In [1]:
import pandas as pd
import numpy as np

In [2]:
Xtrain = pd.read_csv("./Xtrain_hgcGIrA.csv")
Ytrain = pd.read_csv("./Ytrain_yL5OjS4.csv")

Xtest = pd.read_csv("./Xtest.csv")
Ytest = pd.read_csv("./Ysample_iX1anNa.csv")

In [3]:
Xtrain.head()

,date,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,2019-01-07,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.201,0.138,0.091
1,2019-01-08,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.204,0.152,0.106
2,2019-01-10,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.153,0.111
3,2019-01-11,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.152,0.108
4,2019-01-14,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.210,0.147,0.096


In [4]:
print("Shape of Xtrain :" + str(Xtrain.shape))
print("Shape of Ytrain :" + str(Ytrain.shape))

print("Shape of Xtest :" + str(Xtest.shape))
print("Shape of Ytest :" + str(Ytest.shape))


Shape of Xtrain :(31119, 12)
Shape of Ytrain :(31119, 2)
Shape of Xtest :(13752, 12)
Shape of Ytest :(13752, 2)


In [5]:
X = pd.concat([Xtrain, Xtest], axis=0)
X.reset_index(drop=True, inplace=True)



In [6]:
X

,date,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,2019-01-07,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.201,0.138,0.091
1,2019-01-08,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.204,0.152,0.106
2,2019-01-10,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.153,0.111
3,2019-01-11,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.152,0.108
4,2019-01-14,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.210,0.147,0.096
...,...,...,...,...,...,...,...,...,...,...,...,...
44866,2019-07-03,9,0,BE,08:00:00,2,0.121,0.145,0.15715,0.071,0.095,NaN
44867,2019-07-04,9,0,BE,08:00:00,2,0.144,0.139,0.15300,0.079,0.102,NaN
44868,2019-07-05,9,0,BE,08:00:00,2,0.128,0.137,0.12700,0.072,0.089,NaN
44869,2019-05-28,9,0,BE,08:00:00,2,0.164,0.187,0.15500,0.100,0.130,NaN


In [7]:
X['way'].value_counts()

0    44871
Name: way, dtype: int64

In [8]:
X['composition'].value_counts()

2    44860
1       11
Name: composition, dtype: int64

In [9]:
def transform(X):

    columns_to_delete = ['way', 'composition']
    X = X.drop(columns=columns_to_delete, axis=1)
    X['date'] = pd.to_datetime(X['date'], format='%Y-%m-%d')
    X['hour'] = pd.to_datetime(X['hour']).dt.hour
    X['day_of_week'] = X['date'].dt.dayofweek + 1
    X["Train_Station"] = X["train"].astype(str) + X["station"] + X["hour"].astype(str) + X["day_of_week"].astype(str)
    X=X.drop(["train","station", "hour", "day_of_week", "date"], axis=1)
    X = pd.get_dummies(X, columns=["Train_Station"])
    
    return X

In [10]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

imputer = IterativeImputer(estimator=BayesianRidge(), random_state=0)
data = transform(X)
columns = data[['p1q0','p2q0','p3q0','p0q1','p0q2','p0q3']].columns.tolist()
data[['p1q0','p2q0','p3q0','p0q1','p0q2','p0q3']] = pd.DataFrame(imputer.fit_transform(data[['p1q0','p2q0','p3q0','p0q1','p0q2','p0q3']]), columns=columns)

data.head()


c:\Users\admin\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,Train_Station_10AD8.01,Train_Station_10AD8.02,Train_Station_10AD8.03,Train_Station_10AD8.04,...,Train_Station_9BD8.02,Train_Station_9BD8.03,Train_Station_9BD8.04,Train_Station_9BD8.05,Train_Station_9BE7.03,Train_Station_9BE8.01,Train_Station_9BE8.02,Train_Station_9BE8.03,Train_Station_9BE8.04,Train_Station_9BE8.05
0,0.207850,0.214416,0.268776,0.201,0.138,0.091,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.214606,0.220238,0.274366,0.204,0.152,0.106,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.216527,0.221786,0.278743,0.213,0.153,0.111,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.216596,0.221436,0.278083,0.213,0.152,0.108,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.215339,0.219218,0.274190,0.210,0.147,0.096,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = data
X.head()


,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,Train_Station_10AD8.01,Train_Station_10AD8.02,Train_Station_10AD8.03,Train_Station_10AD8.04,...,Train_Station_9BD8.02,Train_Station_9BD8.03,Train_Station_9BD8.04,Train_Station_9BD8.05,Train_Station_9BE7.03,Train_Station_9BE8.01,Train_Station_9BE8.02,Train_Station_9BE8.03,Train_Station_9BE8.04,Train_Station_9BE8.05
0,0.207850,0.214416,0.268776,0.201,0.138,0.091,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.214606,0.220238,0.274366,0.204,0.152,0.106,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.216527,0.221786,0.278743,0.213,0.153,0.111,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.216596,0.221436,0.278083,0.213,0.152,0.108,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.215339,0.219218,0.274190,0.210,0.147,0.096,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
(X.isna().sum()/X.shape[0]).sort_values(ascending=True)

p1q0                      0.0
Train_Station_45AG6.03    0.0
Train_Station_45AG6.04    0.0
Train_Station_45AG6.05    0.0
Train_Station_45AH6.01    0.0
                         ... 
Train_Station_26AQ8.04    0.0
Train_Station_26AQ8.05    0.0
Train_Station_26AS8.01    0.0
Train_Station_26AO8.02    0.0
Train_Station_9BE8.05     0.0
Length: 2950, dtype: float64

In [13]:
Xtrain = X.head(31119)
Xtest = X.tail(len(X) - 31119)

In [14]:
Xtrain

,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,Train_Station_10AD8.01,Train_Station_10AD8.02,Train_Station_10AD8.03,Train_Station_10AD8.04,...,Train_Station_9BD8.02,Train_Station_9BD8.03,Train_Station_9BD8.04,Train_Station_9BD8.05,Train_Station_9BE7.03,Train_Station_9BE8.01,Train_Station_9BE8.02,Train_Station_9BE8.03,Train_Station_9BE8.04,Train_Station_9BE8.05
0,0.207850,0.214416,0.268776,0.201,0.138,0.091000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.214606,0.220238,0.274366,0.204,0.152,0.106000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.216527,0.221786,0.278743,0.213,0.153,0.111000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.216596,0.221436,0.278083,0.213,0.152,0.108000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.215339,0.219218,0.274190,0.210,0.147,0.096000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31114,0.152000,0.188600,0.157000,0.080,0.100,0.090087,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31115,0.153000,0.180400,0.191000,0.089,0.121,0.112281,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31116,0.166000,0.149000,0.168000,0.099,0.129,0.114601,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
31117,0.182000,0.193000,0.162000,0.074,0.101,0.088046,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
Xtest

,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,Train_Station_10AD8.01,Train_Station_10AD8.02,Train_Station_10AD8.03,Train_Station_10AD8.04,...,Train_Station_9BD8.02,Train_Station_9BD8.03,Train_Station_9BD8.04,Train_Station_9BD8.05,Train_Station_9BE7.03,Train_Station_9BE8.01,Train_Station_9BE8.02,Train_Station_9BE8.03,Train_Station_9BE8.04,Train_Station_9BE8.05
31119,0.224159,0.228129,0.288164,0.226,0.165,0.128000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31120,0.222160,0.225268,0.283041,0.221,0.159,0.114000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31121,0.230123,0.232239,0.290719,0.227,0.175,0.133000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31122,0.210966,0.218081,0.272176,0.200,0.148,0.105000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31123,0.226064,0.229108,0.286461,0.221,0.169,0.126000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44866,0.121000,0.145000,0.157150,0.071,0.095,0.090980,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
44867,0.144000,0.139000,0.153000,0.079,0.102,0.093330,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44868,0.128000,0.137000,0.127000,0.072,0.089,0.081021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
44869,0.164000,0.187000,0.155000,0.100,0.130,0.113059,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [16]:
Ytrain

,Unnamed: 0,p0q0
0,1,0.216
1,2,0.216
2,3,0.227
3,4,0.229
4,5,0.225
...,...,...
31114,31115,0.111
31115,31116,0.143
31116,31117,0.139
31117,31118,0.117


In [17]:
Ytrain = Ytrain.drop(["Unnamed: 0"], axis=1)
Ytrain

,p0q0
0,0.216
1,0.216
2,0.227
3,0.229
4,0.225
...,...
31114,0.111
31115,0.143
31116,0.139
31117,0.117


In [22]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Supposons que vous ayez déjà défini vos données Xtrain, Ytrain, Xtest, Ytest

# Créez un pipeline avec une étape de normalisation et une étape SVR
#pipeline = make_pipeline(StandardScaler(), SVR())

# Définissez une plage de valeurs d'hyperparamètres pour SVR (C et epsilon dans cet exemple)
param_grid = {'C': np.logspace(-3, 3, 7), 'epsilon': [0.1, 0.2, 0.5, 1.0]}

svr_model = SVR(kernel='rbf')

# Effectuez une recherche sur grille pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(svr_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(Xtrain, Ytrain)

# Affichez les meilleurs hyperparamètres
best_params = grid_search.best_params_
print(f"Meilleurs hyperparamètres : {best_params}")
  

# Ajustez le modèle avec les meilleurs hyperparamètres
best_svr_model = grid_search.best_estimator_
best_svr_model.fit(Xtrain, Ytrain)

# Évaluez la performance sur l'ensemble de test
predictions = best_svr_model.predict(Xtest)
#mse = mean_squared_error(Ytest, predictions)
#print(f"Mean Squared Error on Test Set: {mse}")


c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [ ]:
#from sklearn.linear_model import RidgeCV
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import numpy as np


#alphas = np.logspace(-6, 6, 13)

#ridge_cv_model = RidgeCV(alphas=alphas, store_cv_values=True)
#ridge_cv_model.fit(Xtrain, Ytrain)

#meilleur_alpha = ridge_cv_model.alpha_
#print(f"Meilleur coefficient alpha : {meilleur_alpha}")

Meilleur coefficient alpha : 1.0


In [19]:
#predictions = ridge_cv_model.predict(Xtest)
predictions = predictions.flatten()

In [20]:
predictions_df = pd.DataFrame({'Predictions': predictions})
predictions_df.index = predictions_df.index + 1
predictions_df.to_csv('predictions.csv', index=True)

In [21]:
predictions_df

,Predictions
1,0.220314
2,0.214177
3,0.225111
4,0.200105
5,0.219271
...,...
13748,0.135056
13749,0.139561
13750,0.133507
13751,0.155036
